In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import casadi as ca
from decision_vars import DecisionVarSet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Test decision vars
from decision_vars import DecisionVarSet
dvs = DecisionVarSet(attr_names = ['lb'], name = 'sneaker/')
dvs.add_vars({'x':[1.0, 2.0]}, lb={'x':[0, 0]})
print(dvs)
dvs2 = DecisionVarSet(attr_names = ['lb'])
dvs2.add_vars(inits={'x':2.0}, lb={'y':3.0})
dvs += dvs2
print(dvs.vectorize('lb'))
d2 = dvs.dictize(np.array([0.1, 0.3, 0.5]))
print(d2)
d = {'test':1}
d.update(dvs2)
print(d)

***** Decision Vars *****
Attributes: ['sym', 'init', 'lb']
Vars: 
  sneaker/x: [x_0, x_1], shape: (2, 1)

[0, 0, -inf]
{'sneaker/x': array([0.1, 0.3]), 'x': array(0.5)}
{'test': 1, 'x': SX(x)}


In [6]:
from robot import *
rob = Robot('config/franka.urdf')

Building robot model from config/franka.urdf with TCP fr3_link8
p:(i0[7])->(o0[3],o1[3x3]) SXFunction


In [7]:
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [0, 10, 10],
        'rest':[0, 0, 0.0]}
c = Contact('friendo', pars)
q = ca.SX.sym('q',7)
p, R = rob.fwd_kin(q)
print(c.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 'R':ca.DM.ones(3)}))

c2 = Contact('c2',pars, ['stiff'], {'cov_init':{}, 'proc_noise':{}})
print(c2.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 
                    'R':ca.DM.ones(3), 
                    'c2/stiff':ca.DM([10, 0, 0])}))

[0, -3, 5]
[-0, -0, 0]


In [27]:
# Test convergence
from robot import Robot
par1 = {'pos': [0, 0, 0.0],
        'stiff': [0, 0, 100],
        'rest':[0, 0, 0.2]}
par2 = {'pos': [0, 0, 0],
        'stiff': [0, 100, 0],
        'rest': [0, 0.5, 0]}
c1 = Contact('c1/', par1, ['stiff'], {'cov_init':{}, 'proc_noise':{}})
c2 = Contact('c2/', par2, [], {'cov_init':{}, 'proc_noise':{}})

rob = Robot('config/franka.urdf', subsys = [c1, c2])
step = rob.build_disc_dyn(0.1)
step_core = rob.disc_dyn_core
# test that it converges as expected 
q = np.zeros(7)
dq = np.zeros(7)
tau = np.zeros(7)
state = {'q':q, 'dq':dq, 'tau_input':tau}
state['c1/stiff'] = ca.DM([0, 0, 1000])
%timeit step(**state)

state_core = {k:v for k,v in state.items()}
state_core['M_inv'] = rob.inv_mass_fn(state['q'])
%timeit step_core(**state)
for _ in range(100):
    res = step(**state)
    state['q'] = res['q_next']
    state['dq'] = res['dq_next']
res_dict = rob.get_ext_state(state)
print(res_dict)
print(res_dict['p'])
#print(res_dict['c1/F'])
#print(res_dict['c2/F'])

Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['c1/', 'c2/']
45.4 µs ± 2.71 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
28.1 µs ± 3.47 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


AttributeError: 'Robot' object has no attribute 'get_extended_state'

In [26]:
# Check that the spawner works
from helper_fns import spawn_models
models = spawn_models('config/franka.yaml', 'config/attrs.yaml', 'config/contact.yaml', sym_vars = ['pos'])
for name, model in models.items():
    step = model.build_disc_dyn(step_size=0.1)
    print(step)
    n_in = step.size_in(0)
    res = step(np.zeros(7), np.zeros(7), np.zeros(3), np.zeros(7))
    print(f"{name} has step fn of {step}")
    print(f"  which evals to {res}")


Building robot model from config/franka.urdf with TCP fr3_link8
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1']
Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['contact_1', 'contact_2']
disc_dyn:(q[7],dq[7],tau_input[7])->(q_next[7],dq_next[7]) SXFunction


RuntimeError: .../casadi/core/function_internal.hpp:1240: Assertion "arg.size()==n_in_" failed:
Incorrect number of inputs: Expected 3, got 4